### Clean and condense Portland 2024 CVRs for city council STV election.
### This notebook: District 4

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True

from tqdm import tqdm

In [2]:
D4_df = pd.read_csv("./Original CVRs/City_of_Portland__Councilor__District_4_2024_11_29_17_26_12.cvr.csv")

In [3]:
D4_df

RowNumber     BoxID  BoxPosition        BallotID  PrecinctID  \
0               1  RCV-0001            1  RCV-0001+10003          26   
1               2  RCV-0001            2  RCV-0001+10005          32   
2               3  RCV-0001            3  RCV-0001+10007          53   
3               4  RCV-0001            4  RCV-0001+10009          22   
4               5  RCV-0001            5  RCV-0001+10011          53   
...           ...       ...          ...             ...         ...   
332964     332965  RCV-1944           26  RCV-1944+10053          53   
332965     332966  RCV-1944           27  RCV-1944+10055          25   
332966     332967  RCV-1944           28  RCV-1944+10057          39   
332967     332968  RCV-1944           29  RCV-1944+10059          70   
332968     332969  RCV-1944           30  RCV-1944+10061          43   

        BallotStyleID PrecinctStyleName ScanComputerName  Status  Remade  \
0                   3            4506-1     ScanStation6       0       0   
1                   1            2804-1     ScanStation6       0       0   
2                   1            3303-1     ScanStation6       0       0   
3                   1            4105-1     ScanStation6       0       0   
4                   1            3303-1     ScanStation6       0       0   
...               ...               ...              ...     ...     ...   
332964              1            3303-1     ScanStation6       0       0   
332965              4            4403-1     ScanStation6       0       0   
332966              2            4707-1     ScanStation6       0       0   
332967              3            4203-1     ScanStation6       0       0   
332968              4            4303-1     ScanStation6       0       0   

        Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON  \
0                                                       0                                             
1                                                       0                                             
2                                                       0                                             
3                                                       0                                             
4                                                       0                                             
...                                                   ...                                             
332964                                                  0                                             
332965                                                  0                                             
332966                                                  0                                             
332967                                                  0                                             
332968                                                  0                                             

        Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON  \
0                                                       0                                             
1                                                       0                                             
2                                                       0                                             
3                                                       0                                             
4                                                       0                                             
...                                                   ...                                             
332964                                                  0                                             
332965                                                  0                                             
332966                                                  0         

### Note that this is all the voters in the city, not just in D4.

In [4]:
list(D4_df.columns)

['RowNumber',
 'BoxID',
 'BoxPosition',
 'BallotID',
 'PrecinctID',
 'BallotStyleID',
 'PrecinctStyleName',
 'ScanComputerName',
 'Status',
 'Remade',
 'Choice_88_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:4:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:5:Number of Winners 3:Bob Weinstein:NON',
 'Choice_88_1:City of Portland, Councilor, District 4:6:Number of Winners 3:Bob Weinstein:NON',
 'Choice_89_1:City of Portland, Councilor, District 4:1:Number of Winners 3:Andra Vltavin:NON',
 'Choice_89_1:City of Portland, Councilor, District 4:2:Number of Winners 3:Andra Vltavin:NON',
 'Choice_89_1:City of Portland, Councilor, District 4:3:Number of Winners 3:Andra

### Here are all the candidates that the D4 CVR contains rankings for:

In [5]:
D4_candidates = [
    'Olivia Clark',
    'Mitch Green',
    'Eric Zimmerman',
    'Eli Arnold',
    'Chad Lykins',
    'Sarah Silkie',
    'Bob Weinstein',
    'Lisa Freeman',
    'Tony Morse',
    'Ben Hufford',
    'Kevin Goldsmith',
    'Andra Vltavin',
    'Stanley Penkin',
    'John Toran',
    'Chloe Mason',
    'Bob Callahan',
    'Moses Ross',
    'Ciatta R Thompson',
    'Raquel Coyote',
    'Mike DiNapoli',
    'John J Goldsmith',
    'Chris Henry',
    'Joseph (Joe) Alfone',
    'Uncertified Write In',
    'Michael Trimble',
    'Kelly Doyle',
    'Brandon Farley',
    'Patrick Cashman',
    'Tony Schwartz',
    'Lee Odell',
    'Christopher Regis',
    'Write-in-129',
    'Write-in-130',
    'Write-in-131'
]

### Collect columns where 1st place, etc. votes are tallied so we can look for overvotes/undervotes while converting to a better format.

In [6]:
rank1_cols = [col for col in D4_df.columns if '1:Number' in col]
rank2_cols = [col for col in D4_df.columns if '2:Number' in col]
rank3_cols = [col for col in D4_df.columns if '3:Number' in col]
rank4_cols = [col for col in D4_df.columns if '4:Number' in col]
rank5_cols = [col for col in D4_df.columns if '5:Number' in col]
rank6_cols = [col for col in D4_df.columns if '6:Number' in col]

all_rank_cols = rank1_cols + rank2_cols + rank3_cols + rank4_cols + rank5_cols + rank6_cols

### This dataset has rows for ALL Portland voters; trim it down to voters who cast at least one vote.

In [7]:
D4_voters_df = D4_df[D4_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True)

### Add a column for each raw ranking:

In [8]:
D4_voters_df.insert(10, "Rank1 raw", None)
D4_voters_df.insert(11, "Rank2 raw", None)
D4_voters_df.insert(12, "Rank3 raw", None)
D4_voters_df.insert(13, "Rank4 raw", None)
D4_voters_df.insert(14, "Rank5 raw", None)
D4_voters_df.insert(15, "Rank6 raw", None)


### For each voter and each rank, record whether the voter overvoted or undervoted in that rank, or if they voted for exactly one candidate in that rank, record that candidate.

In [9]:
for i in tqdm(D4_voters_df.index):
    
    rank1_votes = D4_voters_df.loc[i, rank1_cols].sum()
    if rank1_votes == 0:
        D4_voters_df.loc[i, 'Rank1 raw'] = "skipped"
    elif rank1_votes > 1:
        D4_voters_df.loc[i, 'Rank1 raw'] = "overvote"
    elif rank1_votes == 1:
        this_vote_col = [col for col in rank1_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank1 raw'] = this_vote

    rank2_votes = D4_voters_df.loc[i, rank2_cols].sum()
    if rank2_votes == 0:
        D4_voters_df.loc[i, 'Rank2 raw'] = "skipped"
    elif rank2_votes > 1:
        D4_voters_df.loc[i, 'Rank2 raw'] = "overvote"
    elif rank2_votes == 1:
        this_vote_col = [col for col in rank2_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank2 raw'] = this_vote

    rank3_votes = D4_voters_df.loc[i, rank3_cols].sum()
    if rank3_votes == 0:
        D4_voters_df.loc[i, 'Rank3 raw'] = "skipped"
    elif rank3_votes > 1:
        D4_voters_df.loc[i, 'Rank3 raw'] = "overvote"
    elif rank3_votes == 1:
        this_vote_col = [col for col in rank3_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank3 raw'] = this_vote

    rank4_votes = D4_voters_df.loc[i, rank4_cols].sum()
    if rank4_votes == 0:
        D4_voters_df.loc[i, 'Rank4 raw'] = "skipped"
    elif rank4_votes > 1:
        D4_voters_df.loc[i, 'Rank4 raw'] = "overvote"
    elif rank4_votes == 1:
        this_vote_col = [col for col in rank4_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank4 raw'] = this_vote

    rank5_votes = D4_voters_df.loc[i, rank5_cols].sum()
    if rank5_votes == 0:
        D4_voters_df.loc[i, 'Rank5 raw'] = "skipped"
    elif rank5_votes > 1:
        D4_voters_df.loc[i, 'Rank5 raw'] = "overvote"
    elif rank5_votes == 1:
        this_vote_col = [col for col in rank5_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank5 raw'] = this_vote

    rank6_votes = D4_voters_df.loc[i, rank6_cols].sum()
    if rank6_votes == 0:
        D4_voters_df.loc[i, 'Rank6 raw'] = "skipped"
    elif rank6_votes > 1:
        D4_voters_df.loc[i, 'Rank6 raw'] = "overvote"
    elif rank6_votes == 1:
        this_vote_col = [col for col in rank6_cols if D4_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D4_candidates if name in this_vote_col][0]
        D4_voters_df.loc[i, 'Rank6 raw'] = this_vote


100%|████████████████████████████████████| 77332/77332 [03:07<00:00, 411.55it/s]


### Next, clean the votes for VoteKit according to Portland's rules: overvotes and undervotes are both skipped and ballots are transferred to the next ranking.  If the same candidate receives multiple rankings, only keep the highest ranking.
### It appears that the categories 'Write-in-129', 'Write-in-130', and 'Write-in-131' are ALSO skipped and ballots are transferred to the next ranking.  (Equivalently, these "candidates" were eliminated first in a "Round 0" that occurred before the published tabulation rounds.)

In [10]:
D4_voters_df.insert(16, "Rank1 condensed", None)
D4_voters_df.insert(17, "Rank2 condensed", None)
D4_voters_df.insert(18, "Rank3 condensed", None)
D4_voters_df.insert(19, "Rank4 condensed", None)
D4_voters_df.insert(20, "Rank5 condensed", None)
D4_voters_df.insert(21, "Rank6 condensed", None)


In [11]:
D4_candidates_R1 = D4_candidates[:-3]
D4_candidates_R1

['Olivia Clark',
 'Mitch Green',
 'Eric Zimmerman',
 'Eli Arnold',
 'Chad Lykins',
 'Sarah Silkie',
 'Bob Weinstein',
 'Lisa Freeman',
 'Tony Morse',
 'Ben Hufford',
 'Kevin Goldsmith',
 'Andra Vltavin',
 'Stanley Penkin',
 'John Toran',
 'Chloe Mason',
 'Bob Callahan',
 'Moses Ross',
 'Ciatta R Thompson',
 'Raquel Coyote',
 'Mike DiNapoli',
 'John J Goldsmith',
 'Chris Henry',
 'Joseph (Joe) Alfone',
 'Uncertified Write In',
 'Michael Trimble',
 'Kelly Doyle',
 'Brandon Farley',
 'Patrick Cashman',
 'Tony Schwartz',
 'Lee Odell',
 'Christopher Regis']

In [12]:
condensed_ballots_R1 = []

for i in tqdm(D4_voters_df.index):
    this_ballot_raw = D4_voters_df.loc[i, ['Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw', 'Rank5 raw', 'Rank6 raw']]
    
    this_ballot_cleaned = [name for name in this_ballot_raw if name in D4_candidates_R1]

    this_ballot_condensed = []
    for name in this_ballot_cleaned:
        if name not in this_ballot_condensed:
            this_ballot_condensed.append(name)    

    if len(this_ballot_condensed) > 0:
        condensed_ballots_R1.append(this_ballot_condensed)
    
    if len(this_ballot_condensed) < 6:
        fill_len = 6 - len(this_ballot_condensed)
        this_ballot_condensed = this_ballot_condensed + ["skipped"]*fill_len

    D4_voters_df.loc[i, "Rank1 condensed"] = this_ballot_condensed[0]
    D4_voters_df.loc[i, "Rank2 condensed"] = this_ballot_condensed[1]
    D4_voters_df.loc[i, "Rank3 condensed"] = this_ballot_condensed[2]
    D4_voters_df.loc[i, "Rank4 condensed"] = this_ballot_condensed[3]
    D4_voters_df.loc[i, "Rank5 condensed"] = this_ballot_condensed[4]
    D4_voters_df.loc[i, "Rank6 condensed"] = this_ballot_condensed[5]
    

100%|███████████████████████████████████| 77332/77332 [00:34<00:00, 2218.19it/s]


### Create two datasets to save: one with the raw data and one with the condensed data.  Keep all the intro columns for the raw data, but not for the cleaned data.

In [13]:
D4_voters_df.columns[0:22]

Index(['RowNumber', 'BoxID', 'BoxPosition', 'BallotID', 'PrecinctID',
       'BallotStyleID', 'PrecinctStyleName', 'ScanComputerName', 'Status',
       'Remade', 'Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw',
       'Rank5 raw', 'Rank6 raw', 'Rank1 condensed', 'Rank2 condensed',
       'Rank3 condensed', 'Rank4 condensed', 'Rank5 condensed',
       'Rank6 condensed'],
      dtype='object')

In [14]:
D4_raw_df = D4_voters_df[D4_voters_df.columns[0:16]]

In [15]:
D4_condensed_df = D4_voters_df[['PrecinctStyleName'] + list(D4_voters_df.columns)[16:22]]

### The raw file is good to go; just rename the ranking columns.

In [16]:
D4_raw_df = D4_raw_df.rename(columns = {
    'Rank1 raw' : 'rank1',
    'Rank2 raw' : 'rank2',
    'Rank3 raw' : 'rank3',
    'Rank4 raw' : 'rank4',
    'Rank5 raw' : 'rank5',
    'Rank6 raw' : 'rank6'
})

In [17]:
D4_raw_df.to_csv("./Cleaned CVRs/Portland_D4_Cleaned.csv", index = False)

### For the condensed file, remove voters who no longer have any votes. (These are the ones who only voted for 'Write-in-129', 'Write-in-130', and/or 'Write-in-131'.)

In [18]:
D4_condensed_df = D4_condensed_df[D4_condensed_df['Rank1 condensed'] != 'skipped'].reset_index(drop="True")

### Rename the columns:

In [19]:
D4_condensed_df = D4_condensed_df.rename(columns = {
    'PrecinctStyleName': 'Precinct',
    'Rank1 condensed' : 'rank1',
    'Rank2 condensed' : 'rank2',
    'Rank3 condensed' : 'rank3',
    'Rank4 condensed' : 'rank4',
    'Rank5 condensed' : 'rank5',
    'Rank6 condensed' : 'rank6'
})

### Change the format of the entries in the Precinct column to match the precinct shapefile:

In [20]:
set(D4_condensed_df['Precinct'])

{'2801-1',
 '2803-1',
 '2804-1',
 '2805-1',
 '2806-1',
 '3301-1',
 '3302-1',
 '3303-1',
 '3304-1',
 '3305-1',
 '3306-1',
 '3307-1',
 '3401-1',
 '3803-1',
 '3804-1',
 '3805-1',
 '3806-1',
 '4101-1',
 '4102-1',
 '4103-1',
 '4105-1',
 '4206-2',
 'CLACK-115-2',
 'CLACK-116-1',
 'WASH-366-1',
 'WASH-375-1',
 'WASH-391-1',
 'WASH-392-1',
 'WASH-441-1'}

In [21]:
for i in D4_condensed_df.index:
    if D4_condensed_df.loc[i, 'Precinct'].startswith('CLACK'):
        D4_condensed_df.loc[i, 'Precinct'] = 'C'+ D4_condensed_df.loc[i, 'Precinct'][-5:-2]
    elif D4_condensed_df.loc[i, 'Precinct'].startswith('WASH'):
        D4_condensed_df.loc[i, 'Precinct'] = 'W'+ D4_condensed_df.loc[i, 'Precinct'][-5:-2]
    else:
        D4_condensed_df.loc[i, 'Precinct'] = 'M'+ D4_condensed_df.loc[i, 'Precinct'][0:4]

In [22]:
D4_condensed_df

Precinct           rank1           rank2          rank3           rank4  \
0        M2804  Stanley Penkin    Olivia Clark        skipped         skipped   
1        M3303     Chad Lykins    Sarah Silkie   Lisa Freeman  Eric Zimmerman   
2        M4105    Olivia Clark    Sarah Silkie     John Toran     Mitch Green   
3        M4103     Mitch Green    Sarah Silkie    Chad Lykins         skipped   
4        M2806      Moses Ross    Sarah Silkie  Bob Weinstein  Eric Zimmerman   
...        ...             ...             ...            ...             ...   
76712    M3805     Mitch Green    Sarah Silkie        skipped         skipped   
76713    M3303     Mitch Green     Chris Henry  Andra Vltavin     Chad Lykins   
76714    M3307     Mitch Green   Andra Vltavin    Chad Lykins    Sarah Silkie   
76715    M4206    Lisa Freeman     Chloe Mason        skipped         skipped   
76716    M3303    Olivia Clark  Eric Zimmerman     Eli Arnold    Sarah Silkie   

               rank5         rank6  
0            skipped       skipped  
1       Olivia Clark    Tony Morse  
2       Lisa Freeman  Bob Callahan  
3            skipped       skipped  
4       Olivia Clark  Lisa Freeman  
...              ...           ...  
76712        skipped       skipped  
76713   Sarah Silkie       skipped  
76714    Chris Henry       skipped  
76715        skipped       skipped  
76716  Bob Weinstein   Mitch Green  

[76717 rows x 7 columns]

### Save it:

In [23]:
D4_condensed_df.to_csv("./Cleaned CVRs/Portland_D4_Condensed.csv", index = False)